In [1]:
# Standard Libraries
import os
import math
import json
import datetime
import numpy as np
import time
import nvtx

import argparse
import matplotlib.pyplot as plt

# ------------------------------- CUSTOM FUNCTIONS ------------------------------------------------
# Custom Library
import sys
sys.path.append('../')
    
from proxy_apps.apps.timeseries_prediction import deepDMD, proxyDeepDMD, proxyDeepDMDMGPU, proxyDeepDMDPyTorch, hyperparameters

from proxy_apps.utils.tf import TimingCallback
from proxy_apps.utils.data.main import NpEncoder
from proxy_apps.utils import file_reader, path_handler
from proxy_apps.utils.data.grid import GridNetworkDataHandler, GridNetworkTFDataHandler, GridNetworkNewGen, GridDataGenPyTorch, TransientDataset


import functools
import math
import onnx
import onnxruntime as ort
# from tensorflow.keras.utils import Progbar

In [2]:
# System Setup
config = file_reader.read_config()

_N_EPOCHS = 20
_BATCH_SIZE = 65536
_APP_NAME = config["info"]["app_name"]
_NROWS = int(config["data"]["n_rows"])
_NCOLS = int(config["data"]["n_cols"])
_REPEAT_COLS = int(config["data"]["repeat_cols"])
_WINDOW_SIZE = int(config["data"]["window_size"])
_SHIFT_SIZE = int(config["data"]["shift_size"])
_STRIDE = int(config["data"]["stride"])
_N_SIGNALS = int(config["data"]["n_signals"])

_DTYPE = config["model"]["dtype"]

_MIXED_PRECISION = False
_ONNX_IMPL = False
_INFERENCE_BS = 8192

_N_GPUS = 1
_N_CPUS = 16

_LABEL = "Baseline"
_SUFFIX =  "gpu" + '_' + \
            "a100" + '_' + \
            'ng' + str(_N_GPUS) + '_' + \
            'nc' + str(_N_CPUS) + '_' + \
            'e' + str(_N_EPOCHS) + '_' + \
            'b' + str(_BATCH_SIZE) + '_' + \
            'r' + str(_REPEAT_COLS) + '_' + \
            'mp' + str(0) + '_' + _LABEL

performance_dict = dict()

# current directory
curr_dir = "./"

# output directory
output_dir = path_handler.get_absolute_path(curr_dir, config["info"]["data_dir"] + config["info"]["name"] + "/" + config["info"]["app_name"] + "/" + _DTYPE + "/R" + str(_REPEAT_COLS) + "/")
model_dir = path_handler.get_absolute_path(curr_dir, config["model"]["model_dir"] + config["info"]["name"] + "/" + config["info"]["app_name"] + "/" + _DTYPE + "/R" + str(_REPEAT_COLS) + "/")

hyper_param_dict = config["model"]["hyperparameters"]
hyper_param_dict['original_dim']       = _REPEAT_COLS * _NCOLS   # input data dimension
hyper_param_dict['num_epochs']         = _N_EPOCHS  # Number of epochs  
hyper_param_dict['batch_size']         = _BATCH_SIZE

hyper_param_dict['dtype']         = _DTYPE
hp = hyperparameters.HyperParameters(hyper_param_dict)
hp.model_name         = _LABEL

performance_dict["n_epochs"] = hp.ep
performance_dict["batch_size"] = hp.bs

In [3]:
if _LABEL == "PyTorch":
    import torch
    print("[INFO] PyTorch version: ", torch.__version__)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    
    if _DTYPE == "float64": torch.set_default_dtype(torch.float64)
    else: torch.set_default_dtype(torch.float32)
        
    def get_indexer(n_rows, window_size, shift_size, start_point, leave_last):
        return np.arange(window_size)[None, :] + start_point + shift_size*np.arange(((n_rows - window_size - leave_last - start_point) // shift_size) + 1)[:, None]
else:   
    import tensorflow as tf
    import tf2onnx

    # TensorFlow Setup
    print("[INFO] Tensorflow version: ", tf.__version__)
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        print("Name:", gpu.name, "  Type:", gpu.device_type)

    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    for gpu in logical_gpus:
        print("Name:", gpu.name, "  Type:", gpu.device_type)

    # tf.compat.v1.disable_eager_execution()
    print("[INFO] Eager mode: ", tf.executing_eagerly()) # For easy reset of notebook state.

    # Setup TensorFlow
    tf.keras.backend.clear_session()
    tf.config.optimizer.set_jit(True) # Enable XLA.

    # Setup Precision
    if _LABEL in ["Baseline"]: 
        tf.keras.backend.set_floatx('float64')
    elif _LABEL in ["TFDataGen", "TFDataOptMGPU", "TFDataOptMGPUAcc"]:
        tf.keras.backend.set_floatx(_DTYPE)
        if _MIXED_PRECISION:
            # set floatx
            _DTYPE = 'float32'
            hp.d_type         = _DTYPE
            tf.keras.backend.set_floatx(_DTYPE)
            # set policy
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
            # check dtypes
            print('Compute dtype: %s' % policy.compute_dtype)
            print('Variable dtype: %s' % policy.variable_dtype)

    # Mirror Strategy for MGPUs
    if _LABEL in ["TFDataOptMGPU", "TFDataOptMGPUAcc"]:
        mirrored_strategy = tf.distribute.MirroredStrategy()

    # To avoid GPU Congestion
    os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

    @tf.function(experimental_compile=True)
    def get_indexer(n_rows, window_size, shift_size, start_point, leave_last):
        return np.arange(window_size)[None, :] + start_point + shift_size*np.arange(((n_rows - window_size - leave_last - start_point) // shift_size) + 1)[:, None]

[INFO] Tensorflow version:  2.4.0
[INFO] Eager mode:  True


In [10]:
if _LABEL in ["TFDataGen", "TFDataOptMGPU", "TFDataOptMGPUAcc"]:
    class DataGenerator(tf.keras.utils.Sequence):
        'Generates data for Keras'
        def __init__(self, file_lists, n_rows, n_cols, repeat_cols):
            'Initialization'
            self.n_rows = n_rows 
            self.n_cols = n_cols
            self.repeat_cols = repeat_cols 
            self.file_lists = file_lists

        def __len__(self):
            'Denotes the number of batches per epoch'
            return len(self.file_lists)

        def __getitem__(self, index):
            'Generate one batch of data'
            file_name = self.file_lists[index].decode('utf-8')
            dataset = TransientDataset(file_name)
            raw_data = np.repeat(np.concatenate([dataset.F, dataset.Vm], axis=1), self.repeat_cols, axis=1)[:self.n_rows, :]


            return raw_data
elif _LABEL == "PyTorch":    
    class GridDataGenPyTorch(torch.utils.data.Dataset):
        'Characterizes a dataset for PyTorch'
        def __init__(self, dir_list, n_rows, n_cols, repeat_cols, d_type="float64"):
            'Initialization'
            self.n_rows = n_rows 
            self.n_cols = n_cols
            self.repeat_cols = repeat_cols 
            self.file_lists = dir_list

        def __len__(self):
            'Denotes the total number of samples'
            return len(self.file_lists)

        def __getitem__(self, index):
            'Generate one batch of data'
            file_name = self.file_lists[index]
            dataset = TransientDataset(file_name)
            raw_data = np.repeat(np.concatenate([dataset.F, dataset.Vm], axis=1), self.repeat_cols, axis=1)[:self.n_rows, :]


            return raw_data

In [5]:
# ------------------------------- CREATE DATA HANDLER ------------------------------------------------   

data_handler_nvtx = nvtx.start_range("Create Data Handler")
dh_start = time.time()    
inp_data_dir = config["info"]["input_dir"]
if _LABEL in ["Baseline", "TFDataOpt"]:
    data_handler = GridNetworkDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, inp_data_dir),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            dtype=_DTYPE
                                         ) 

    scenario_data = data_handler.load_grid_data()

elif _LABEL in ["TFDataGen", "TFDataOptMGPU", "TFDataOptMGPUAcc"]:
    if _ONNX_IMPL:
        data_handler = GridNetworkDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, inp_data_dir),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            dtype=_DTYPE
                                         ) 

        scenario_data = data_handler.load_grid_data()
    else:
        scenario_dir=path_handler.get_absolute_path(curr_dir, inp_data_dir)
        DIR_LIST = [scenario_dir + "/" + f + "/" for f in os.listdir(scenario_dir)]

        scenario_data = tf.data.Dataset.from_generator(
            DataGenerator,
            output_signature = tf.TensorSpec(shape=(_NROWS, _NCOLS*_REPEAT_COLS), dtype=_DTYPE),
            args=(DIR_LIST, _NROWS, _NCOLS, _REPEAT_COLS)
        ).prefetch(tf.data.AUTOTUNE)
    
elif _LABEL == "PyTorch":
    if _ONNX_IMPL:
        data_handler = GridNetworkDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, inp_data_dir),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            dtype=_DTYPE
                                         ) 

        scenario_data = data_handler.load_grid_data()
    else:
        scenario_dir=path_handler.get_absolute_path(curr_dir, inp_data_dir)
        DIR_LIST = [scenario_dir + "/" + f + "/" for f in os.listdir(scenario_dir)]

        test_data = GridDataGenPyTorch(DIR_LIST, _NROWS, _NCOLS, _REPEAT_COLS)
        scenario_data = torch.utils.data.DataLoader(test_data, pin_memory=True, num_workers=int(_N_CPUS))    

dh_stop = time.time()
performance_dict['data_handler_time'] = dh_stop-dh_start
nvtx.end_range(data_handler_nvtx)

[INFO]: Loading the datasets from the directory: /qfs/people/jain432/pacer/data/TrainingDataIEEE68bus
[INFO]: Loading data for 30 scenarios ...
[INFO]: Loaded 30/30 scenarios ...
[INFO]: Total number of scenarios loaded: 30
[INFO]: Shape of each scenario loaded:  (1400, 1360)
[INFO]: Done ...


In [6]:
path_handler.get_absolute_path(model_dir, _SUFFIX)

'/qfs/people/jain432/pacer/models/deepDMD/power_systems/scenarios_30/float64/R10/gpu_a100_ng1_nc16_e20_b65536_r10_mp0_Baseline'

In [7]:
if _LABEL == "Baseline":
    K_model = tf.keras.models.load_model(path_handler.get_absolute_path(model_dir, _SUFFIX))
elif _LABEL in ["TFDataGen", "TFDataOptMGPU", "TFDataOptMGPUAcc"]:
    K_model = tf.keras.models.load_model(path_handler.get_absolute_path(model_dir, _SUFFIX))
elif _LABEL in ["PyTorch"]:
    K_model = torch.load(path_handler.get_absolute_path(model_dir, _SUFFIX), map_location=device)

In [8]:
scale_factor = 2 * np.pi
split_index = (_NCOLS * _REPEAT_COLS) // 2
freq_pred_error_list = []
vol_pred_error_list = []

inf_time_start = time.time()
for X_actual in scenario_data:
    if _LABEL == "Baseline": X_scaled = X_actual.copy()
    elif _LABEL in ["TFDataGen", "TFDataOptMGPU", "TFDataOptMGPUAcc"]: X_scaled = X_actual.numpy().copy()
    elif _LABEL in ["PyTorch"]: X_scaled = X_actual.clone()

    # normalize the data
    X_scaled[:, :split_index]  = scale_factor * (X_actual[:, :split_index] - 60)
    X_scaled[:, split_index:] = 10 * (X_actual[:, split_index:] - 1)

    if _LABEL == "Baseline": Psi_X = K_model.encoder(X_scaled, training=False)
    elif _LABEL in ["TFDataGen", "TFDataOptMGPU", "TFDataOptMGPUAcc"]: Psi_X = K_model.predict(X_scaled)
    elif _LABEL in ["PyTorch"]: Psi_X = K_model(X_scaled, training=False)
    
    PSI_X = np.concatenate([X_scaled, Psi_X], 1)
    # 1-time step evolution on observable space:
    if _LABEL in ["Baseline", "TFDataGen", "TFDataOptMGPU", "TFDataOptMGPUAcc"]: prediction = np.matmul(PSI_X, K_model.KO)

    true_freq = X_actual[1:, :68]
    pred_freq = 60 + (1 / scale_factor) * prediction[:-1, :68]
    freq_pred_error = np.mean(np.sqrt(np.mean(np.square(true_freq - pred_freq), axis=1)))
    freq_pred_error_list.append(freq_pred_error)

    true_voltage = X_actual[1:, split_index:split_index+68]
    pred_voltage = 1 + 0.1 * prediction[:-1, split_index:split_index+68]
    vol_pred_error = np.mean(np.sqrt(np.mean(np.square(true_voltage - pred_voltage), axis=1)))
    vol_pred_error_list.append(vol_pred_error)

inf_time_stop = time.time()

performance_dict["inference_time"] = inf_time_stop - inf_time_start
performance_dict["freq_pred_error_list"] = freq_pred_error_list
performance_dict["vol_pred_error_list"] = vol_pred_error_list

In [9]:
print(performance_dict["inference_time"])

2.0445399284362793


In [9]:
performance_dict

{'n_epochs': 20,
 'batch_size': 65536,
 'data_handler_time': 2.433096170425415,
 'inference_time': 2.1665444374084473,
 'freq_pred_error_list': [0.0017837500431607204,
  0.002015474373534101,
  0.001326770826793299,
  0.001547627046527474,
  0.001539102598161702,
  0.0017185856230452824,
  0.0013867048285448713,
  0.0020397569384255185,
  0.0013782706867285373,
  0.0010624184672534968,
  0.0016774282659270704,
  0.003820516321498636,
  0.0014378449358107166,
  0.002778635669051693,
  0.0018501006699941857,
  0.0021398790892613465,
  0.002286480462990819,
  0.0025621045305733925,
  0.0011289677927891822,
  0.0023509873851126606,
  0.0024587049813889483,
  0.001653458627223324,
  0.0010227442899055173,
  0.0017683292030398084,
  0.0020103323149494793,
  0.0016789675424866402,
  0.0018844534816607065,
  0.002434259175482385,
  0.0012220338660044665,
  0.0018787555045888043],
 'vol_pred_error_list': [0.0011327691271397116,
  0.001274261355950784,
  0.0008626086454031818,
  0.00101288427544

In [9]:
X_scaled

array([[-1.41652882e-01, -1.41652882e-01, -1.41652882e-01, ...,
        -1.58421639e-02, -1.58421639e-02, -1.58421639e-02],
       [-1.35115266e-01, -1.35115266e-01, -1.35115266e-01, ...,
        -1.48963824e-02, -1.48963824e-02, -1.48963824e-02],
       [-1.28491166e-01, -1.28491166e-01, -1.28491166e-01, ...,
        -1.39310853e-02, -1.39310853e-02, -1.39310853e-02],
       ...,
       [ 9.24009223e-05,  9.24009223e-05,  9.24009223e-05, ...,
         8.98589917e-04,  8.98589917e-04,  8.98589917e-04],
       [ 9.28583402e-05,  9.28583402e-05,  9.28583402e-05, ...,
         9.00263545e-04,  9.00263545e-04,  9.00263545e-04],
       [ 9.29531219e-05,  9.29531219e-05,  9.29531219e-05, ...,
         9.01881198e-04,  9.01881198e-04,  9.01881198e-04]])

In [10]:
performance_dict

{'n_epochs': 20,
 'batch_size': 65536,
 'data_handler_time': 0.0407257080078125,
 'inference_time': 5.200047016143799,
 'freq_pred_error_list': [0.0014600613662991793,
  0.0017246852913969071,
  0.0010693021129632454,
  0.0012190540515727736,
  0.0012960760961770138,
  0.00137911755560042,
  0.001013679697186699,
  0.0018820301599848479,
  0.0010790285905788112,
  0.0007333914920327088,
  0.0012922767341083012,
  0.003810049737820211,
  0.0011986593566978723,
  0.00266590261551473,
  0.0014686439488587892,
  0.0018194525738775157,
  0.0018256582644738803,
  0.0022800239321146072,
  0.0008107272947972389,
  0.0020161307383703875,
  0.002216905452836981,
  0.0014760134402501265,
  0.0007140356299137225,
  0.0015564652865930984,
  0.0017955624059643053,
  0.0014115319936165065,
  0.001692411672481672,
  0.002252034618612325,
  0.0009173261320526966,
  0.0014946968175355796],
 'vol_pred_error_list': [0.0008925092132816285,
  0.0010778056973166268,
  0.0006505850344706889,
  0.0007698925415

In [11]:
performance_dict

{'n_epochs': 20,
 'batch_size': 65536,
 'data_handler_time': 2.4671921730041504,
 'inference_time': 1.9216105937957764,
 'freq_pred_error_list': [0.0017837500431607204,
  0.002015474373534101,
  0.001326770826793299,
  0.001547627046527474,
  0.001539102598161702,
  0.0017185856230452824,
  0.0013867048285448713,
  0.0020397569384255185,
  0.0013782706867285373,
  0.0010624184672534968,
  0.0016774282659270704,
  0.003820516321498636,
  0.0014378449358107166,
  0.002778635669051693,
  0.0018501006699941857,
  0.0021398790892613465,
  0.002286480462990819,
  0.0025621045305733925,
  0.0011289677927891822,
  0.0023509873851126606,
  0.0024587049813889483,
  0.001653458627223324,
  0.0010227442899055173,
  0.0017683292030398084,
  0.0020103323149494793,
  0.0016789675424866402,
  0.0018844534816607065,
  0.002434259175482385,
  0.0012220338660044665,
  0.0018787555045888043],
 'vol_pred_error_list': [0.0011327691271397116,
  0.001274261355950784,
  0.0008626086454031818,
  0.0010128842754

In [9]:
performance_dict

{'n_epochs': 20,
 'batch_size': 65536,
 'data_handler_time': 0.03106236457824707,
 'inference_time': 5.2708213329315186,
 'freq_pred_error_list': [array([0.01092259, 0.01064888, 0.01040552, ..., 0.00052688, 0.00052739,
         0.00052787]),
  array([0.00821148, 0.00902034, 0.01005057, ..., 0.00040037, 0.0004015 ,
         0.00040261]),
  array([0.00521915, 0.00521514, 0.00517879, ..., 0.00051594, 0.00051677,
         0.00051759]),
  array([0.00971645, 0.00953831, 0.00936281, ..., 0.00050598, 0.00050572,
         0.00050549]),
  array([0.00672803, 0.00658259, 0.00637579, ..., 0.00052756, 0.00052829,
         0.00052904]),
  array([0.00948891, 0.009786  , 0.01000539, ..., 0.00047266, 0.00047254,
         0.00047242]),
  array([0.00663309, 0.00659982, 0.00654394, ..., 0.00051769, 0.00051774,
         0.00051781]),
  array([0.00852792, 0.00835993, 0.00827513, ..., 0.00033436, 0.00033629,
         0.00033821]),
  array([0.00681933, 0.00689289, 0.00689104, ..., 0.0004929 , 0.0004935 ,
     

In [50]:
freq_pred_error = np.sqrt(np.mean(np.square(true_freq - pred_freq), axis=1))

[array([0.00059382]), array([0.00061265]), array([0.0005362]), array([0.00047053]), array([0.0006054]), array([0.00052865]), array([0.00049522]), array([0.00074062]), array([0.00050548]), array([0.00054475]), array([0.0004811]), array([0.00104868]), array([0.00052173]), array([0.00095544]), array([0.00050758]), array([0.00059822]), array([0.00053966]), array([0.00050199]), array([0.00054626]), array([0.0006212]), array([0.00074212]), array([0.0004887]), array([0.00055189]), array([0.00055942]), array([0.00070505]), array([0.0005665]), array([0.00056296]), array([0.00047138]), array([0.00051297]), array([0.00052941])]


In [40]:
np.mean(np.square(true_freq - pred_freq), axis=1)

array([7.74068147e-07])

In [46]:
np.sqrt(np.mean(np.square(true_freq - pred_freq), axis=1))

array([0.00087981])

In [7]:
class GridNetworkSequentialGen():
    def __init__(self, scenario_dir, n_rows, n_cols, d_type, repeat_cols=1):
        self.scenario_dir = scenario_dir
        self.n_rows = n_rows 
        self.n_cols = n_cols
        self.repeat_cols = repeat_cols 
        self.d_type = d_type

    #@tf.function(experimental_compile=True)
    def load_grid_data(self):
        # load data and print list of files
        # print('[INFO]: Loading the datasets from the directory:', self.scenario_dir)
        self.dir_list = [self.scenario_dir + "/" + f + "/" for f in os.listdir(self.scenario_dir)]
        
        gen = SequentialGenerator(self.dir_list, self.n_rows, self.repeat_cols)
        dataset_gen = tf.data.Dataset.from_generator(
            lambda: gen,
            output_types=self.d_type,
            output_shapes=(self.n_rows, self.n_cols*self.repeat_cols)
        ).prefetch(tf.data.AUTOTUNE)
        
        trimmed_scenarios = tf.data.Dataset.range(1).interleave(
            lambda x: dataset_gen,
            num_parallel_calls=tf.data.AUTOTUNE).cache()
        
        return trimmed_scenarios
    
    def get_training_data(self, trimmed_scenarios, window_size, shift_size, stride, N, y_start):
        training_data = trimmed_scenarios.interleave(lambda x: TrainingDataGenerator(x, 
                                                                                     self.n_cols, 
                                                                                     self.repeat_cols, 
                                                                                     self.d_type,
                                                                                     self.n_rows-N, 
                                                                                     y_start, 
                                                                                     window_size, 
                                                                                     shift_size, 
                                                                                     stride), num_parallel_calls=tf.data.AUTOTUNE)
        return training_data

In [67]:
scenario_dir=path_handler.get_absolute_path(curr_dir, inp_data_dir)
dir_list = [scenario_dir + "/" + f + "/" for f in os.listdir(scenario_dir)]

In [69]:
dg = DataGenerator(dir_list, 1400, 136, 10)

In [73]:
freq_pred_error_list = []
vol_pred_error_list = []

inf_time_start = time.time()
for X_actual in dg:
    X = X_actual[500:502, :]
    X_scaled = X.copy()

    # normalize the data
    scale_factor = 2*np.pi
    X_scaled[:, :1360//2]  = scale_factor * (X[:, :1360//2] - 60)
    X_scaled[:, 1360//2:] = 10 * (X[:, 1360//2:] - 1)

    Psi_X = K_model.encoder(X_scaled[:1, :], training=False)
    PSI_X = tf.concat([X_scaled[:1, :], tf.cast(Psi_X, hp.d_type)], 1)
    # 1-time step evolution on observable space:
    prediction = np.matmul(PSI_X, K_model.KO)

    true_freq = X[1:2, :68]
    pred_freq = 60 + (1 / scale_factor) * prediction[:, :68]
    freq_pred_error = np.sqrt(np.mean(np.square(true_freq - pred_freq), axis=1))
    freq_pred_error_list.append(freq_pred_error)

    true_voltage = X[1:2, (1360//2):(1360//2)+68]
    pred_voltage = 1 + 0.1 * prediction[:, (1360//2):(1360//2)+68]
    vol_pred_error = np.sqrt(np.mean(np.square(true_voltage - pred_voltage), axis=1))
    vol_pred_error_list.append(vol_pred_error)

inf_time_stop = time.time()

In [71]:
for i in dg:
    print(i.shape)

(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)
(1400, 1360)


In [13]:
performance_dict

{'n_epochs': 20,
 'batch_size': 65536,
 'data_handler_time': 2.495760679244995,
 'inference_time': 0.14789938926696777,
 'freq_pred_error_list': [0.0008125413080226986,
  0.0008011221807706098,
  0.0007930578393291258,
  0.000861956136772595,
  0.000983705293880578,
  0.0008569141555937215,
  0.0008631455395613538,
  0.0009937617542325596,
  0.0007601961052588285,
  0.0009089427301325305,
  0.0008728779322401594,
  0.0015818447054952113,
  0.0007302367417507845,
  0.001203431405415618,
  0.0008808599998479264,
  0.0008300696165020215,
  0.0009795352230041008,
  0.0008868098049478076,
  0.0008333678105909688,
  0.0008243481715125153,
  0.001032714959226045,
  0.0007160653682456894,
  0.0008802648192627636,
  0.0007715697875265792,
  0.0009443095940439503,
  0.0008682021315725761,
  0.0007416884173349355,
  0.0007741767018055888,
  0.000771973490301609,
  0.0008798114267965387],
 'vol_pred_error_list': [0.0005938154345194646,
  0.0006126452664008093,
  0.0005361967691988196,
  0.00047052

In [11]:
print(freq_pred_error_list, vol_pred_error_list)

[0.0008125413080226986, 0.0008011221807706098, 0.0007930578393291258, 0.000861956136772595, 0.000983705293880578, 0.0008569141555937215, 0.0008631455395613538, 0.0009937617542325596, 0.0007601961052588285, 0.0009089427301325305, 0.0008728779322401594, 0.0015818447054952113, 0.0007302367417507845, 0.001203431405415618, 0.0008808599998479264, 0.0008300696165020215, 0.0009795352230041008, 0.0008868098049478076, 0.0008333678105909688, 0.0008243481715125153, 0.001032714959226045, 0.0007160653682456894, 0.0008802648192627636, 0.0007715697875265792, 0.0009443095940439503, 0.0008682021315725761, 0.0007416884173349355, 0.0007741767018055888, 0.000771973490301609, 0.0008798114267965387] [0.0005938154345194646, 0.0006126452664008093, 0.0005361967691988196, 0.0004705270218975176, 0.0006054026638161656, 0.0005286544951903434, 0.0004952156032895991, 0.0007406181736292537, 0.0005054765375479886, 0.0005447456045183207, 0.00048110301010281364, 0.0010486825920767151, 0.0005217325839178682, 0.00095544231

In [ ]:
# # performance_dict["inference_size"] = Yp_array.shape[0]
# performance_dict["inference_time"] = inf_time_stop - inf_time_start
# performance_dict["freq_pred_error_list"] = freq_pred_error_list
# performance_dict["vol_pred_error_list"] = vol_pred_error_list
    
#     inf_time_start = time.time()
#     for X_actual in scenario_data:
#         X_scaled = X_actual.numpy()

#         # normalize the data
#         X_scaled[:, :split_index]  = scale_factor * (X_actual[:, :split_index] - 60)
#         X_scaled[:, split_index:] = 10 * (X_actual[:, split_index:] - 1)

#         # create predictions
#         Psi_X = encoder.predict(X_scaled)
#         PSI_X = tf.concat([X_scaled, tf.cast(Psi_X, hp.d_type)], 1)

#         # 1-time step evolution on observable space:
#         prediction = np.matmul(PSI_X, encoder.KO)

#         true_freq = X_actual[1:, :68]
#         pred_freq = 60 + (1 / scale_factor) * prediction[:-1, :68]
#         freq_pred_error = np.sqrt(np.mean(np.square(true_freq - pred_freq), axis=1))
#         freq_pred_error_list.append(freq_pred_error)

#         true_voltage = X_actual[1:, split_index:split_index+68]
#         pred_voltage = 1 + 0.1 * prediction[:-1, split_index:split_index+68]
#         vol_pred_error = np.sqrt(np.mean(np.square(true_voltage - pred_voltage), axis=1))
#         vol_pred_error_list.append(vol_pred_error)

#     inf_time_stop = time.time()

#     # performance_dict["inference_size"] = Yp_array.shape[0]